In [27]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import gradio as gr


In [28]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set
Google API Key exists and begins AIzaSyA0


In [29]:
openai= OpenAI()
google.generativeai.configure()
MODEL ='gpt-4o-mini'

In [34]:
system_message = 'you are a helpful assistant'

In [35]:
def chat_gpt(message,history):
    messages=[{"role":"system","content":system_message}]+history+[{"role":"user","content":message}]

    stream =openai.chat.completions.create(model=MODEL,messages=messages,stream=True)


    response =""
    for chunk in stream:
        response+=chunk.choices[0].delta.content or ''
        yield response

In [38]:
def chat_gemini(message, history):
    """Chat with Gemini model using messages format"""
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-2.0-flash-exp',
        system_instruction=system_message
    )
    
    # Convert history to Gemini format
    # Extract only 'role' and 'content' from history messages
    messages = []
    for msg in history:
        role = msg.get("role", "")
        content = msg.get("content", "")
        
        if role == "user":
            messages.append({"role": "user", "parts": [content]})
        elif role == "assistant":
            messages.append({"role": "model", "parts": [content]})
    
    # Add the current user message
    messages.append({"role": "user", "parts": [message]})
    
    response = gemini.generate_content(messages, stream=True)
    result = ""
    for chunk in response:
        if chunk.candidates and chunk.candidates[0].content.parts:
            text = chunk.candidates[0].content.parts[0].text
            result += text
            yield result

In [46]:
def chat(message, history, model):
    """Main chat function for gr.ChatInterface"""
    if model == "GPT-4o-mini":
        yield from chat_gpt(message, history)
    elif model == "Gemini 2.0 Flash":
        yield from chat_gemini(message, history)
    else:
        yield "Please select a model"


# Using Blocks for custom accordion label
with gr.Blocks(title="Multi-Model Chat") as demo:
    gr.Markdown("# Multi-Model Chat")
    gr.Markdown("WHERE SHOULD WE BEGIN ?")
    
    chatbot = gr.Chatbot(type='messages', label="Chat with me 💕")
    
    with gr.Accordion("MODEL", open=False):
        model_dropdown = gr.Dropdown(
            choices=["GPT-4o-mini", "Gemini 2.0 Flash"],
            value="GPT-4o-mini",
            label="Select Model",
            container=False
        )
    
    msg = gr.Textbox(
        placeholder="Type a message...",
        container=False,
        scale=7
    )
    
    def respond(message, chat_history, model):
        # Add user message to history
        chat_history.append({"role": "user", "content": message})
        
        # Stream bot response
        bot_response = ""
        for response in chat(message, chat_history[:-1], model):
            bot_response = response
            # Update chat history with streaming response
            updated_history = chat_history[:-1] + [
                {"role": "user", "content": message},
                {"role": "assistant", "content": bot_response}
            ]
            yield "", updated_history
    
    msg.submit(respond, [msg, chatbot, model_dropdown], [msg, chatbot])

demo.launch()

* Running on local URL:  http://127.0.0.1:7887
* To create a public link, set `share=True` in `launch()`.


In [43]:
def chat(message, history, model):
    """Main chat function for gr.ChatInterface"""
    if model == "GPT-4o-mini":
        yield from chat_gpt(message, history)
    elif model == "Gemini 2.0 Flash":
        yield from chat_gemini(message, history)
    else:
        yield "Please select a model"


# Using Blocks for custom accordion label
with gr.Blocks(title="Multi-Model Chat") as demo:
    gr.Markdown("# Multi-Model Chat")
    gr.Markdown("WHERE SHOULD WE BEGIN ?")
    
    chatbot = gr.Chatbot(type='messages', label="Chat with me 💕")
    
    with gr.Accordion("MODEL", open=False):
        model_dropdown = gr.Dropdown(
            choices=["GPT-4o-mini", "Gemini 2.0 Flash"],
            value="GPT-4o-mini",
            label="Select Model",
            container=False
        )
    
    msg = gr.Textbox(
        placeholder="Type a message...",
        container=False,
        scale=7
    )
    
    def respond(message, chat_history, model):
        chat_history.append({"role": "user", "content": message})
        bot_response = ""
        for response in chat(message, chat_history[:-1], model):
            bot_response = response
        chat_history.append({"role": "assistant", "content": bot_response})
        return "", chat_history
    
    msg.submit(respond, [msg, chatbot, model_dropdown], [msg, chatbot])

demo.launch()

* Running on local URL:  http://127.0.0.1:7884
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "D:\gen-ai\anaconda\envs\llms\Lib\site-packages\httpx\_transports\default.py", line 101, in map_httpcore_exceptions
    yield
  File "D:\gen-ai\anaconda\envs\llms\Lib\site-packages\httpx\_transports\default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\gen-ai\anaconda\envs\llms\Lib\site-packages\httpcore\_sync\connection_pool.py", line 256, in handle_request
    raise exc from None
  File "D:\gen-ai\anaconda\envs\llms\Lib\site-packages\httpcore\_sync\connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\gen-ai\anaconda\envs\llms\Lib\site-packages\httpcore\_sync\connection.py", line 101, in handle_request
    raise exc
  File "D:\gen-ai\anaconda\envs\llms\Lib\site-packages\httpcore\_sync\connection.py", line 78, in handle_request
    stream = self._connect(request)
           

In [40]:
def chat(message, history, model):
    """Main chat function for gr.ChatInterface"""
    if model == "GPT-4o-mini":
        yield from chat_gpt(message, history)
    elif model == "Gemini 2.0 Flash":
        yield from chat_gemini(message, history)
    else:
        yield "Please select a model"


# ChatInterface with model selection dropdown
gr.ChatInterface(
    fn=chat,
    type='messages',
    title="Multi-Model Chat",
    description="Chat with me❤️",
    additional_inputs=[
        gr.Dropdown(
            choices=["GPT-4o-mini", "Gemini 2.0 Flash"],
            value="GPT-4o-mini",
            label="Select Model"
        )
    ]
).launch()

* Running on local URL:  http://127.0.0.1:7881
* To create a public link, set `share=True` in `launch()`.


In [18]:
def stream_model(message,model):
    if model =="GPT":
        result= chat_gpt(message)
    elif model =="gemini":
        result =chat_gemini(message)
    else:
        raise ValueError("Unknown model")
    yield from result

### Gradio integration

In [6]:
gr.ChatInterface(fn=chat,type='messages').launch()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.
